In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

s = requests.Session()

In [ ]:
r = s.get('https://ceouttarpradesh.nic.in/rollpdf/rollpdf.aspx', headers=headers)
r.status_code

In [ ]:
soup = BeautifulSoup(r.content)

select_elem = soup.find("select", {"name": "ctl00$ContentPlaceHolder1$DDLDistrict"})

options = []
for option_elem in select_elem.find_all("option"):
    option_value = option_elem["value"]
    option_text = option_elem.text
    options.append((option_value, option_text))

districts = options[1:]
districts

In [ ]:
rows = []

In [ ]:
for d in districts:
    print(d)
    r = s.get('https://ceouttarpradesh.nic.in/rollpdf/rollpdf.aspx', headers=headers)
    print(r.status_code)
    soup = BeautifulSoup(r.content)            
    viewstategenerator = soup.find('input', {'id': '__VIEWSTATEGENERATOR'})['value']
    eventvalidation = soup.find('input', {'id': '__EVENTVALIDATION'})['value']
    viewstate = soup.select_one('#aspnetForm input[name*="__VIEWSTATE"]')['value']
    data = {'__EVENTTARGET': 'ctl00$ContentPlaceHolder1$DDLDistrict',
            '__EVENTARGUMENT': '',
            '__LASTFOCUS': '',
            '__VIEWSTATE': viewstate,
            '__VIEWSTATEGENERATOR': viewstategenerator,
            '__EVENTVALIDATION': eventvalidation,
            'ctl00$ContentPlaceHolder1$DDLDistrict': d[0],
           }
    r = s.post('https://ceouttarpradesh.nic.in/rollpdf/rollpdf.aspx', headers=headers, data=data)
    print(r.status_code)
    soup = BeautifulSoup(r.content)
    select_elem = soup.find("select", {"name": "ctl00$ContentPlaceHolder1$DDL_AC"})
    options = []
    for option_elem in select_elem.find_all("option"):
        option_value = option_elem["value"]
        option_text = option_elem.text
        options.append((option_value, option_text))

    acs = options[1:]
    print(acs)
    first = True
    for ac in acs:
        print(ac)
        if not first:
            r = s.get('https://ceouttarpradesh.nic.in/rollpdf/rollpdf.aspx', headers=headers)
            print(r.status_code)
            soup = BeautifulSoup(r.content)
            viewstategenerator = soup.find('input', {'id': '__VIEWSTATEGENERATOR'})['value']
            eventvalidation = soup.find('input', {'id': '__EVENTVALIDATION'})['value']
            viewstate = soup.select_one('#aspnetForm input[name*="__VIEWSTATE"]')['value']
            data = {'__EVENTTARGET': 'ctl00$ContentPlaceHolder1$DDLDistrict',
                    '__EVENTARGUMENT': '',
                    '__LASTFOCUS': '',
                    '__VIEWSTATE': viewstate,
                    '__VIEWSTATEGENERATOR': viewstategenerator,
                    '__EVENTVALIDATION': eventvalidation,
                    'ctl00$ContentPlaceHolder1$DDLDistrict': d[0],
                   }
            r = s.post('https://ceouttarpradesh.nic.in/rollpdf/rollpdf.aspx', headers=headers, data=data)
            print(r.status_code)
            soup = BeautifulSoup(r.content)
        first = False
        viewstategenerator = soup.find('input', {'id': '__VIEWSTATEGENERATOR'})['value']
        eventvalidation = soup.find('input', {'id': '__EVENTVALIDATION'})['value']
        viewstate = soup.select_one('#aspnetForm input[name*="__VIEWSTATE"]')['value']
        data = {'__EVENTTARGET': '',
                '__EVENTARGUMENT': '',
                '__LASTFOCUS': '',
                '__VIEWSTATE': viewstate,
                '__VIEWSTATEGENERATOR': viewstategenerator,
                '__EVENTVALIDATION': eventvalidation,
                'ctl00$ContentPlaceHolder1$DDLDistrict': d[0],
                'ctl00$ContentPlaceHolder1$DDL_AC': ac[0],
                'ctl00$ContentPlaceHolder1$Button1': 'Show'
               }
        r = s.post('https://ceouttarpradesh.nic.in/rollpdf/rollpdf.aspx', headers=headers, data=data)
        print(r.status_code)
        soup = BeautifulSoup(r.content)
        table = soup.find('table', {'id': 'ctl00_ContentPlaceHolder1_ElecRollGrd'})
        a = table.find_all('a')[-1]
        m = re.findall('Page\$(\d+)', a.attrs['href'])
        if m:
            trs = table.find_all('tr')[1:-2]
        else:
            trs = table.find_all('tr')[1:]
        for tr in trs:
            cols = tr.find_all('td')
            row = {'District': d[1],
                   'AC_Name': ac[1],
                   'AC_Number': cols[0].text,
                   'Part_Number': cols[1].text,
                   'Polling_Station_Name': cols[2].text,
                   'ElectorRollPdf': 'https://ceouttarpradesh.nic.in/rollpdf/' + cols[4].find('a').attrs['href']
                  }
            rows.append(row)
        if len(m) == 0:
            continue
        p = 2
        while True:
            viewstategenerator = soup.find('input', {'id': '__VIEWSTATEGENERATOR'})['value']
            eventvalidation = soup.find('input', {'id': '__EVENTVALIDATION'})['value']
            viewstate = soup.select_one('#aspnetForm input[name*="__VIEWSTATE"]')['value']
            data = {'__EVENTTARGET': 'ctl00$ContentPlaceHolder1$ElecRollGrd',
                    '__EVENTARGUMENT': 'Page$%d' % p,
                    '__LASTFOCUS': '',
                    '__VIEWSTATE': viewstate,
                    '__VIEWSTATEGENERATOR': viewstategenerator,
                    '__EVENTVALIDATION': eventvalidation,
                    'ctl00$ContentPlaceHolder1$DDLDistrict': d[0],
                    'ctl00$ContentPlaceHolder1$DDL_AC': ac[0],
                   }
            r = s.post('https://ceouttarpradesh.nic.in/rollpdf/rollpdf.aspx', headers=headers, data=data)
            #print(p, r.status_code)
            soup = BeautifulSoup(r.content)
            table = soup.find('table', {'id': 'ctl00_ContentPlaceHolder1_ElecRollGrd'})
            for tr in table.find_all('tr')[1:-2]:
                cols = tr.find_all('td')
                row = {'District': d[1],
                       'AC_Name': ac[1],
                       'AC_Number': cols[0].text,
                       'Part_Number': cols[1].text,
                       'Polling_Station_Name': cols[2].text,
                       'ElectorRollPdf': 'https://ceouttarpradesh.nic.in/rollpdf/' + cols[4].find('a').attrs['href']
                      }
                rows.append(row)
            a = table.find_all('a')[-1]
            m = re.findall('Page\$(\d+)', a.attrs['href'])
            last_page = int(m[0])
            p += 1
            if p > last_page:
                break
        #break
    #break

In [ ]:
df = pd.DataFrame(rows)
df

In [ ]:
df.drop_duplicates(inplace=True)
df

In [ ]:
df.to_csv('data/up-2023-electoral-rolls.csv', index=False)